In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from IPython import display
from tqdm.notebook import tqdm
import random
import math, time, os
from matplotlib import pyplot as plt
import pickle as pkl

from AutoEncoder import Autoencoder as AE
from copy import deepcopy
from sklearn.model_selection import train_test_split

device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
prefix = './data_dump'


In [2]:
# day checker
file_paths = ['../Datasets/df_train_shuffled.csv', 
              '../Datasets/df_val.csv',
              '../Datasets/df_val_test.csv']

for file in file_paths:
#     print("NEW FILE")
    df = pd.read_csv(file)
    print(file, df.shape)
#     print(np.unique(df['day_no'].value_counts().to_numpy()))

    # cur_day = df['day_no'][0]
    # cur_era = df['era'][0]
    # for i in range(1, df.shape[0]):
    #     if df['day_no'][i] == cur_day and df['era'][i] == cur_era:
    #         pass
    #     elif df['day_no'][i] == cur_day and df['era'][i] != cur_era:
    #         print("1")
    #     elif df['day_no'][i] != cur_day and df['era'][i] == cur_era:
    #         print("2")
    #     else:
    #         pass
    #     cur_era = df['era'][i]
    #     cur_day = df['day_no'][i] 

../Datasets/df_train_shuffled.csv (175127, 28)
../Datasets/df_val.csv (52212, 28)
../Datasets/df_val_test.csv (56802, 28)


In [32]:
class SinusodialDataset(Dataset):
    def __init__(self, df, test=False):
        """ creating label columns of eras and targets """
        self.NUM_FEATURES = 24
        self.X = df.iloc[:, :self.NUM_FEATURES]
        self.testMode = test
        if self.testMode == False:
            self.y = df['target_10_val']
        self.X = self.create_categorical_one_hot(self.X)
    
    def create_categorical_one_hot(self, df):
        categories = [0, 0.25, 0.5, 0.75, 1]
        one_hot_encoded_columns = []
        for col in df.columns:
            for cat in categories:
                new_col_name = f"{col}_{cat}"
                one_hot_encoded_col = (df[col] == cat).astype(int)
                one_hot_encoded_col.name = new_col_name
                one_hot_encoded_columns.append(one_hot_encoded_col)

        # Concatenate the one-hot encoded columns along axis 1
        return pd.concat(one_hot_encoded_columns, axis=1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
        if self.testMode == False:
            y = torch.tensor(self.y.iloc[idx], dtype=torch.long)
            return X, y
        return X

In [33]:
def make_data_splits(train_df, val_df, test_df, batch_size=32):

    def encode(v, class_values):
        return class_values.index(v)

    class_values = train_df['target_10_val'].unique().tolist()
    train_df['target_10_val'] = train_df['target_10_val'].apply(lambda x: encode(x, class_values))
    train_df.reset_index(drop=True, inplace=True)

    data_train = SinusodialDataset(train_df)
    loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=False)
    loader_val = None
    loader_test = None

    if val_df is not None:
        class_values = val_df['target_10_val'].unique().tolist()
        val_df['target_10_val'] = val_df['target_10_val'].apply(lambda x: encode(x, class_values))
        val_df.reset_index(drop=True, inplace=True)

        data_val = SinusodialDataset(val_df)
        loader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False)

    if test_df is not None:
        class_values = test_df['target_10_val'].unique().tolist()
        test_df['target_10_val'] = test_df['target_10_val'].apply(lambda x: encode(x, class_values))
        test_df.reset_index(drop=True, inplace=True)
            
        data_test = SinusodialDataset(test_df)
        loader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False)

    print("Train-val-test lengths: ", len(data_train), len(data_val), len(data_test))

    return loader_train, loader_val, loader_test

In [34]:
class TestTimeAdapter(nn.Module):
    
    def __init__(self, ae_dims, cl_dims, lr=1e-3, weight_decay=1e-3):
        super(TestTimeAdapter,self).__init__()
        self.ae_dims=ae_dims
        self.cl_dims = cl_dims

        self.ae = AE(ae_dims)
        self.classifier=nn.ModuleList()
        
        for i in range(len(cl_dims)-2):
            self.classifier.append(nn.Linear(cl_dims[i],cl_dims[i+1]))
            self.classifier.append(nn.ReLU())
        self.classifier.append(nn.Linear(cl_dims[i+1],cl_dims[i+2]))
        self.classifier.append(nn.LogSoftmax(dim=1))
        self.optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

    def forward(self, x, classifier=False):
        if classifier:
            x = x.float()
            x = self.ae.encode(x)
            for l in self.classifier:
                x = l(x)
            return x
        
        x = x.float()
        x = self.ae(x)
        return x

In [35]:
def accuracy(y_pred, y_test, verbose=True):
    m = y_test.shape[0]
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose:
        print(correct,m)
    accuracy = correct/m
    return accuracy, correct

In [36]:
def Test(net, loader_test, \
         device='cpu', Loss=nn.NLLLoss(reduction='sum'), \
         test_time_epochs = 3):
    net.train()
    total_samples = 0
    correct_samples = 0
    loss = 0.0
    step=0
    (X_prev, y_prev) = (None, None)
    (X_prev_prev, y_prev_prev) = (None, None)
    # test_mode is False
    for (X, y) in loader_test:
        # print(step, end=" ")
        X=X.to(device)
        y=y.to(device)
        total_samples += y.shape[0]
   
        for e in range(test_time_epochs): 
            x_reconst = net(X, classifier=False)
            ae_loss = 0.0
            for feat in range(0, X.shape[-1], 5):
                ae_loss += Loss(nn.LogSoftmax(dim=1)(x_reconst[:, feat:feat+5]),X[:, feat:feat+5].argmax(dim=1))
            net.optimizer.zero_grad()
            ae_loss.backward()
            net.optimizer.step()
        
            if X_prev_prev is not None:
                y_pred = net(X_prev_prev, classifier=True)
                cl_loss = Loss(y_pred, y_prev_prev)
                net.optimizer.zero_grad()
                cl_loss.backward()
                net.optimizer.step()

        y_pred = net(X, classifier=True)
        cl_loss = Loss(y_pred, y)

        loss += (ae_loss+cl_loss).item()
        _, i_cor_sam = accuracy(y_pred, y,verbose=False)
        correct_samples += i_cor_sam
        step+=1

        (X_prev_prev, y_prev_prev) = (X_prev, y_prev)
        (X_prev, y_prev) = (X, y)

    # print()
    
    acc = correct_samples / total_samples
    loss /= total_samples
    print('Test/Val loss:', loss, 'Test/Val acc:', acc)
    return loss, acc

In [37]:
def Train(Net, train_loader, epochs=20, Loss=nn.NLLLoss(reduction='sum'), 
          verbose=False, device='cpu',
          val_ds=None, loader_test=None):
    model_save_time = time.time()
    losses = []
    accs = []
    val_losses=[]
    val_accL=[]
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss=0.0
        start_time = time.time()
        correct_samples = 0
        total_samples = 0
        # test_mode = False
        for (X,y) in train_loader:
            X=X.to(device)
            y=y.to(device)
            total_samples += y.shape[0]
            x_reconst = Net(X, classifier=False) # B x nd x nc = B x 24 x 5
            ae_loss = 0.0
            for feat in range(0, X.shape[-1], 5):
                # print((x_reconst[:, feat:feat+5]),X[:, feat:feat+5].argmax(dim=1))
                # print((x_reconst[:, feat:feat+5]).shape,X[:, feat:feat+5].argmax(dim=1).shape)
                ae_loss += Loss(nn.LogSoftmax(dim=1)(x_reconst[:, feat:feat+5]),X[:, feat:feat+5].argmax(dim=1))
            Net.optimizer.zero_grad()
            ae_loss.backward()
            Net.optimizer.step()

            y_pred = Net(X, classifier=True)
            cl_loss = Loss(y_pred, y)
            Net.optimizer.zero_grad()
            cl_loss.backward()
            Net.optimizer.step()

            step+=1
            tot_loss+=(ae_loss+cl_loss)
            if verbose:
                _, i_cor_sam = accuracy(y_pred, y,verbose=False)
                correct_samples += i_cor_sam
            
        end_time = time.time()
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        a = correct_samples/total_samples
        accs += [a]

        if verbose:
            print('Epoch %2d Loss: %2.5e Accuracy: %2.5f Epoch Time: %2.5f' %(e,l,a,t))

        val_loss, val_acc = Test(deepcopy(Net), val_ds, device = device)
        val_losses.append(val_loss)
        val_accL.append(val_acc)

        # print("TESTING BUDDY:")
        # Test(deepcopy(Net), loader_test, device=device)

        torch.save(Net.state_dict(), f'{prefix}/net_{str(model_save_time)}.pth')

    return Net, losses, accs, val_losses, val_accL

In [38]:
def plot_loss_acc(losses, accs, val_losses, val_accs):

    plt.plot(np.array(accs),color='red', label='Train accuracy')
    plt.plot(np.array(val_accs),color='blue', label='Val accuracy')
    plt.legend()
    plt.savefig(f'{prefix}/acc_tta.png')
    plt.clf()

    plt.plot(np.array(losses),color='red', label='Train loss')
    plt.plot(np.array(val_losses),color='blue', label='Val loss')
    plt.legend()
    plt.savefig(f'{prefix}/loss_TTA.png')
    plt.clf()
    return

In [48]:
df_paths = ['../Datasets/df_train_shuffled.csv', 
              '../Datasets/df_val.csv',
              '../Datasets/df_val_test.csv']

batch_size = 140

train_df = pd.read_csv(df_paths[0])
val_df = pd.read_csv(df_paths[1])
test_df = pd.read_csv(df_paths[2])

loader_train, loader_val, loader_test = make_data_splits(train_df, val_df, test_df, batch_size=batch_size)
net = TestTimeAdapter(ae_dims=[120, 64, 32, 16, 32, 64, 120], 
                        cl_dims=[16, 16, 5], ).to(device)
net, losses, accs, val_losses, val_accL = Train(net, loader_train, \
                                epochs=20, verbose=True, device=device, \
                                    val_ds=loader_val,loader_test=loader_test)
plot_loss_acc(losses, accs, val_losses, val_accL)
#Testing code
test_loss, test_acc = Test(deepcopy(net), loader_test, device=device)
print(test_loss, test_acc)

Train-val-test lengths:  175127 52212 56802
Epoch  0 Loss: 1.71056e+01 Accuracy: 0.26179 Epoch Time: 24.21161
Test/Val loss: 12.69470003958471 Test/Val acc: 0.28390025281544473
Epoch  1 Loss: 1.14282e+01 Accuracy: 0.27719 Epoch Time: 21.28432
Test/Val loss: 10.732439854022124 Test/Val acc: 0.2877882479123573
Epoch  2 Loss: 9.74013e+00 Accuracy: 0.27921 Epoch Time: 19.09744
Test/Val loss: 9.91403279603692 Test/Val acc: 0.28918639393242934
Epoch  3 Loss: 9.01222e+00 Accuracy: 0.27961 Epoch Time: 19.88013
Test/Val loss: 9.38526553533401 Test/Val acc: 0.28631349115145943
Epoch  4 Loss: 8.50728e+00 Accuracy: 0.27980 Epoch Time: 22.40702
Test/Val loss: 9.057438463395028 Test/Val acc: 0.289626905692178
Epoch  5 Loss: 8.22294e+00 Accuracy: 0.28082 Epoch Time: 20.54733
Test/Val loss: 8.854935711739593 Test/Val acc: 0.292748793380832
Epoch  6 Loss: 8.07467e+00 Accuracy: 0.28035 Epoch Time: 19.96255
Test/Val loss: 8.702982484102524 Test/Val acc: 0.2954301693097372
Epoch  7 Loss: 7.94375e+00 Accur

RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 128 vs 0

In [ ]:
def TestTime(net, loader_test, \
         device='cpu', Loss=nn.NLLLoss(reduction='sum'), \
         test_time_epochs = 3):
    net.train()
    total_samples = 0
    loss = 0.0
    step=0
    
    # test_mode is True
    for X in loader_test:
        X=X.to(device)
        
        total_samples += X.shape[0]
   
        for e in range(test_time_epochs): 
            x_reconst = net(X, classifier=False)
            ae_loss = 0.0
            for feat in range(0, X.shape[-1], 5):
                ae_loss += Loss(nn.LogSoftmax(dim=1)(x_reconst[:, feat:feat+5]),X[:, feat:feat+5].argmax(dim=1))
            net.optimizer.zero_grad()
            ae_loss.backward()
            net.optimizer.step()

        loss += (ae_loss).item()
        step+=1
    
    loss /= total_samples
    print('Test/Val loss:', loss)
    return loss

In [ ]:
import os

def encode(v, class_values):
        return class_values.index(v)


def list_csv_files(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    return csv_files

for date in [2,3,4,5,8]:
    file_path = f'../Datasets/live_data_0{date}-Apr-2024'
    csv_files = list_csv_files(file_path)

    round_dict = {}

    for i in range(len(csv_files)):
        round = int(csv_files[i].split(".")[0].split("_")[-1])
        if round not in round_dict.keys():
            round_dict[round] = {'train': '', 'test': ''}
        if 'train' in csv_files[i]:
            round_dict[round]['train'] = file_path + '/' + csv_files[i]
        else:
            round_dict[round]['test'] = file_path + '/' + csv_files[i]
            

    results = pd.DataFrame()

    for round in round_dict.keys():
        print('Working on - ', round_dict[round]['train'])
        df = pd.read_csv(round_dict[round]['train'])
        class_values = df['target_10_val'].unique().tolist()
        df['target_10_val'] = df['target_10_val'].apply(lambda x: encode(x, class_values))
        df.reset_index(drop=True, inplace=True)
        data = SinusodialDataset(df, test=False)
        loader_adapt = DataLoader(data, batch_size=1, shuffle=False)
        adapt_loss, adapt_acc = Test(net, loader_adapt, device=device, test_time_epochs=2)
        
        print('Working on - ', round_dict[round]['test'])
        df = pd.read_csv(round_dict[round]['test'])

        ids = df['id']
        df = df.drop(columns=['id'])
        row_num = df['row_num']
        
        data = SinusodialDataset(df, test=True)
        loader_adapt = DataLoader(data, batch_size=1, shuffle=False)
        adapt_loss = TestTime(net, loader_adapt, device=device)

        predictions = []

        total_samples = 0
        for X in loader_adapt:
            X=X.to(device)
            total_samples += X.shape[0]
            y_pred = net(X, classifier=True)
            predictions.append(y_pred.argmax(dim=1).detach().cpu().numpy())
            print(predictions)

        temp_df = pd.DataFrame()
        temp_df['id'] = ids
        temp_df['predictions'] = predictions
        temp_df['row_num'] = row_num
        temp_df['round_no'] = [f'{round}']*ids.shape[0]
        results = pd.concat([results, temp_df])
    
    
    results.to_csv(f'predictions_0{date}-04-2024.csv')


Working on -  ../Datasets/live_data_02-Apr-2024/df_live_train_02-Apr-2024_129.csv
Test/Val loss: 25.671604070907986 Test/Val acc: 0.2846153846153846
Working on -  ../Datasets/live_data_02-Apr-2024/df_live_test_02-Apr-2024_129.csv
Test/Val loss: 21.708186467488606
[array([2])]
[array([2]), array([4])]
[array([2]), array([4]), array([2])]
Working on -  ../Datasets/live_data_02-Apr-2024/df_live_train_02-Apr-2024_122.csv
Test/Val loss: 24.17695481215066 Test/Val acc: 0.2601626016260163
Working on -  ../Datasets/live_data_02-Apr-2024/df_live_test_02-Apr-2024_122.csv
Test/Val loss: 13.774435997009277
[array([0])]
Working on -  ../Datasets/live_data_02-Apr-2024/df_live_train_02-Apr-2024_143.csv
Test/Val loss: 22.293966385170265 Test/Val acc: 0.18518518518518517
Working on -  ../Datasets/live_data_02-Apr-2024/df_live_test_02-Apr-2024_143.csv
Test/Val loss: 22.326815923055012
[array([0])]
[array([0]), array([0])]
[array([0]), array([0]), array([0])]
Working on -  ../Datasets/live_data_02-Apr-20

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = Test(net, loader_val, device=device)
print(test_loss, test_acc)

In [ ]:
class MLP(nn.Module):
    def __init__(self, dims, task='classification', lr=1e-2, weight_decay=0):
        super(MLP,self).__init__()
        self.dims=dims
        self.task=task
        self.layers=nn.ModuleList()
        for i in range(len(self.dims)-2):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
        self.layers.append(nn.LogSoftmax(dim=1))
        self.optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

    def forward(self,x):
        x = x.float()
        for l in self.layers:
            x = l(x)
        return x
    

def accuracy(Net, X_test, y_test, verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose:
        print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy, correct


def Test(net, loader_test, \
         device='cpu', Loss=nn.NLLLoss(reduction='sum')):
    net.eval()
    total_samples = 0
    correct_samples = 0
    loss = 0.0
    for (X, y) in loader_test:
        X=X.to(device)
        y=y.to(device)
        total_samples += y.shape[0]
        _, i_cor_sam = accuracy(net,X,y,verbose=False)
        correct_samples += i_cor_sam
        loss += Loss(net(X), y).cpu().detach().item()
    acc = correct_samples / total_samples
    loss /= total_samples
    return loss, acc

def Train(Net, data, mode, noise_level, epochs=20, lr=5e-2, Loss=nn.NLLLoss(reduction='sum'), verbose=False, device='cpu',
          val_ds=None, plot_accs=False, plot_losses=False):
    model_save_time = time.time()
    losses = []
    accs = []
    val_losses=[]
    val_accL=[]
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss=0.0
        start_time = time.time()
        correct_samples = 0
        total_samples = 0
        for (X,y) in data:
            X=X.to(device)
            y=y.to(device)
            total_samples += y.shape[0]
            y_pred = Net(X)
            loss = Loss(y_pred,y)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
            if verbose:
                _, i_cor_sam = accuracy(Net,X,y,verbose=False)
                correct_samples += i_cor_sam
        end_time = time.time()
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        a = correct_samples/total_samples
        accs += [a]

        if verbose:
            print('Epoch %2d Loss: %2.5e Accuracy: %2.5f Epoch Time: %2.5f' %(e,l,a,t))

        val_loss, val_acc = Test(Net, val_ds, mode, noise_level, device)
        val_losses.append(val_loss)
        val_accL.append(val_acc)

        # Net.eval()
        # if plot_accs and val_ds is not None:
        #     val_total_samples = 0
        #     val_correct_samples = 0
        #     for (X, y) in val_ds:
        #       X=X.to(device)
        #       y=y.to(device)
        #       val_total_samples += y.shape[0]
        #       _, i_val_cor_sam = accuracy(Net,X,y,verbose=False)
        #       val_correct_samples += i_val_cor_sam
        #     val_accL+=[val_correct_samples / val_total_samples]
        #     plt.plot(np.array(val_accL),color='red')
        #     plt.plot(np.array(accs),color='blue')
        #     plt.show()

        # if plot_losses and val_ds is not None:
        #     val_loss_ = 0.0
        #     val_total_samples = 0
        #     for (X, y) in val_ds:
        #       X=X.to(device)
        #       y=y.to(device)
        #       val_total_samples += y.shape[0]
        #       val_loss_ += Loss(Net(X), y).cpu().detach().item()
        #     val_losses+=[val_loss_/val_total_samples]
        #     plt.plot(val_losses,color='red')
        #     plt.plot(losses,color='blue')
        #     plt.show()

        torch.save(Net.state_dict(), f'{prefix}/net_{noise_level}_{mode}_{str(model_save_time)}.pth')

    return Net, losses, accs, val_losses, val_accL

In [ ]:
df_paths = ['../Datasets/df_train_shuffled.csv', 
              '../Datasets/df_val.csv',
              '../Datasets/df_val_test.csv']

batch_size = 140


train_df = pd.read_csv(df_paths[0])
val_df = pd.read_csv(df_paths[1])
test_df = pd.read_csv(df_paths[2])

loader_train, loader_val, loader_test = make_data_splits(train_df, val_df, test_df, batch_size=batch_size)
net = MLP([24, 32, 5], lr=1e-3).to(device)
net, losses, accs, val_losses, val_accL = Train(net, loader_train, '', '',\
                                epochs=20, verbose=True, device=device, \
                                    val_ds=loader_val)
plot_loss_acc(losses, accs, val_losses, val_accL)
#Testing code
test_loss, test_acc = Test(deepcopy(net), loader_test, device=device)
print(test_loss, test_acc)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Gate(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(Gate, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        return F.softmax(self.fc(x), dim=1)

class MixtureOfExperts(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_experts):
        super(MixtureOfExperts, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim, output_dim) for _ in range(num_experts)])
        self.gate = Gate(input_dim, num_experts)
        self.softmax = nn.LogSoftmax(dim=1)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)

    def forward(self, x):
        gate_output = self.gate(x)
        expert_outputs = [expert(x) for expert in self.experts]
        expert_outputs = torch.stack(expert_outputs, dim=1)  # (batch_size, num_experts, output_dim)
        prediction = torch.sum(gate_output.unsqueeze(2) * expert_outputs, dim=1)  # (batch_size, output_dim)
        output = self.softmax(prediction)
        return output

# Example usage
if __name__ == "__main__":
    
    df_paths = ['../Datasets/df_train_shuffled.csv', 
              '../Datasets/df_val.csv',
              '../Datasets/df_val_test.csv']

    batch_size = 140


    train_df = pd.read_csv(df_paths[0])
    val_df = pd.read_csv(df_paths[1])
    test_df = pd.read_csv(df_paths[2])

    input_dim = 24
    hidden_dim = 8
    output_dim = 5
    num_experts = 5

    loader_train, loader_val, loader_test = make_data_splits(train_df, val_df, test_df, batch_size=batch_size)
    net = MixtureOfExperts(input_dim, hidden_dim, output_dim, num_experts).to(device)
    net, losses, accs, val_losses, val_accL = Train(net, loader_train, '', '',\
                                    epochs=20, verbose=True, device=device, \
                                        val_ds=loader_val)
    plot_loss_acc(losses, accs, val_losses, val_accL)
    #Testing code
    test_loss, test_acc = Test(deepcopy(net), loader_test, device=device)
    print(test_loss, test_acc)